# About
**Goal:** Demonstrate extraction, transformation, and load use-cases for nonprofit/foundations.

**Actions:**

- Collect social media data on two foundations.
- Transform data into a human readable format 
- Visualize a trend from the data.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import json
import time
import math
import numpy as np

from tweepy import Cursor
from twitter_client import get_twitter_client
from dotenv import load_dotenv, find_dotenv
from etl import usage, paginate, extract, generate_dfs

from highcharts import Highchart
from visualization import options

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [2]:
foundations = ['Asia_Foundation', 'RobinHoodNYC']

In [3]:
# extract(foundations[0])

# Robin Hood Foundation

### [Fighting Poverty in New York](https://www.robinhood.org/)

**Mission:** 
Robin Hood’s mission is to improve the living standards for 1.8 million low-income new yorkers.

<img src='robinhood.png' >

#### Robin Hood Foundation's Following and Followers

In [4]:
rh_following, rh_followers = generate_dfs('RobinHoodNYC')
print("Following: {}".format(rh_following.shape))
print("Followers: {}".format(rh_followers.shape)) # Note the failure rate.

Following: (956, 15)
Followers: (14999, 15)


#### Preview Output

In [5]:
rh_followers.head()

,created_at,description,favourites_count,followers_count,friends_count,id,listed_count,location,name,profile_image_url,relationship,screen_name,source,statuses_count,url
0,Sat Feb 11 13:45:06 +0000 2017,,70,14,350,830412344727629825,0,,haythem kenway,http://pbs.twimg.com/profile_images/8410089605...,follower,marmadukemax,RobinHoodNYC,4,None
1,Thu May 11 13:55:16 +0000 2017,I am very simple.......,26,3,86,862667425539649536,0,"Assam, India",Arif Ahmed,http://pbs.twimg.com/profile_images/8630467814...,follower,ArifAhm08967631,RobinHoodNYC,5,None
2,Sun May 14 16:05:43 +0000 2017,college,2,7,70,863787416909467653,0,,sasi,http://pbs.twimg.com/profile_images/8638014582...,follower,sasig2602,RobinHoodNYC,2,None
3,Fri Jul 04 16:11:08 +0000 2008,"I've been off twitter for a while, but I sudde...",1297,98,322,15318695,0,,Olyenka,http://pbs.twimg.com/profile_images/4540825017...,follower,olyenka,RobinHoodNYC,1392,None
4,Sun May 14 15:00:46 +0000 2017,,3,1,8,863771073661349889,0,,Pedro Gamer 12387,http://abs.twimg.com/sticky/default_profile_im...,follower,P12387,RobinHoodNYC,3,None


# Asia Foundation

### [Nonprofit international development organization committed to improving lives across a dynamic and developing Asia.](http://asiafoundation.org/)

**Mission:** 
The Asia Foundation improves lives, expands opportunities, and helps societies flourish across a dynamic and developing Asia. We work with innovative leaders and communities to build effective institutions and advance pathbreaking reforms. Together with our partners, we are committed to Asia’s continued development as a peaceful, just, and thriving region of the world.

<img src='asiafoundation.png' >

#### Asia Foundation's Following and Followers

In [6]:
af_following, af_followers = generate_dfs('Asia_Foundation')
print("Following: {}".format(af_following.shape))
print("Followers: {}".format(af_followers.shape)) # Note the failure rate.

Following: (2710, 15)
Followers: (14999, 15)


#### Preview Output

In [7]:
af_followers.head()

,created_at,description,favourites_count,followers_count,friends_count,id,listed_count,location,name,profile_image_url,relationship,screen_name,source,statuses_count,url
0,Tue May 27 16:29:48 +0000 2014,"Political Activist, @rammadhavbjp Fan Big Fan ...",10001,49973,5417,2527805598,62,India,Rammadhav,http://pbs.twimg.com/profile_images/8286330937...,follower,RamMadhav4BJP,Asia_Foundation,11706,https://t.co/x9Z840AgfN
1,Mon Jul 01 12:03:17 +0000 2013,★ Malay Organization Independent ★,33897,1635,130,1560359496,84,#Sumatra,★ Melayu Monitor ★,http://pbs.twimg.com/profile_images/8630398005...,follower,MelayuMonitor,Asia_Foundation,37052,https://t.co/Sw5u4dhhzT
2,Fri Apr 22 19:41:16 +0000 2016,"Adventure, travel, people. Media Coordinator, ...",253,69,302,723597556832309248,0,Singapore,Amber Mizerak,http://pbs.twimg.com/profile_images/7236053216...,follower,Amber_Mizerak,Asia_Foundation,45,None
3,Sun May 14 20:08:08 +0000 2017,politics,3,6,143,863848421781667841,0,"Dhaka, Bangladesh",sm soruj,http://pbs.twimg.com/profile_images/8638497358...,follower,sm36576220,Asia_Foundation,2,None
4,Mon Aug 11 17:17:05 +0000 2014,,2,36,174,2724359317,0,,michael schiffer,http://abs.twimg.com/sticky/default_profile_im...,follower,eachinhisprison,Asia_Foundation,105,None


In [8]:
#### Save the Output
af_followers.to_csv("asia_foundation_followers.csv", index=False)
af_following.to_csv("asia_foundation_following.csv", index=False)

# Visualize

In [9]:
def prep_data(x, y, data, name):
    d = data[[x, y]].fillna(1)
    frame = d.values.tolist()
    return {'name': name, 'data': frame}

def plot_scatter(group1, group2):
    H = Highchart(width=850, height=400)
    H.set_dict_options(options)
    H.add_data_set(group1['data'], 'scatter', group1['name'], color='rgba(223, 83, 83, .5)')
    H.add_data_set(group2['data'], 'scatter', group2['name'], color='rgba(119, 152, 191, .5)')
    return H

# Following Profile

In [10]:
af = prep_data('followers_count', 'friends_count', af_following, 'Asia Foundation')
rh = prep_data('followers_count', 'friends_count', rh_following, 'Robin Hood Foundation')
plot_scatter(af, rh)